# Guided Exercise: Performance

### Goals 🎯
In this tutorial, you will use TruEra to train and ingest a model, and then make performance improvements to our model in a structured and methodical way!

In this tutorial, you will:
1. Set up and view the results of performance and feature importance tests.
2. Find actionable issues with the model.
3. Mitigate these issues and re-upload your model to TruEra.
4. Retest the new model and confirm the effectivenesss of the mitigation strategy.

### First, set the credentials for your TruEra deployment.

If you don't have credentials yet, get them instantly by signing up for free at: https://www.truera.com

In [ ]:
# connection details
TRUERA_URL = "https://app.truera.net/"
AUTH_TOKEN = "..."

### Install required packages

In [ ]:
! pip install --upgrade xgboost==1.6.2
! pip install truera

### From here, run the rest of the notebook and follow the analysis.

### Import packages, set up your TruEra Workspace

In [ ]:
import logging

import pandas as pd
import xgboost as xgb
from truera.client.ingestion import ColumnSpec, ModelOutputContext
from truera.client.truera_authentication import TokenAuthentication
from truera.client.truera_workspace import TrueraWorkspace

auth = TokenAuthentication(AUTH_TOKEN)
tru = TrueraWorkspace(TRUERA_URL, auth)

### Load the data and train an xgboost model
A bit about the model and data... 

In this example, we will use real data on the AirBnb listings 🏠 in San Francisco and Seattle to predict the listing price. The Airbnb data was scraped by Inside Airbnb and hosted by OpenDataSoft. Pricing a rental property is a challenging task for Airbnb owners as they need to understand the market, the features of their property, and how those features contribute to listing price.

You can find more information about the data here:
https://data.opendatasoft.com/explore/dataset/airbnb-listings%40public/

In [ ]:
# load data
san_francisco = pd.read_csv(
    'https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-performance/San_Francisco.csv')
seattle = pd.read_csv('https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-performance/Seattle.csv')

# make all float
san_francisco = san_francisco.astype(float).reset_index(names="id")
seattle = seattle.astype(float).reset_index(names="id")

### Now we can create our first project

In [ ]:
# create the first project
project_name = "Starter Example Companion - Performance"

tru.add_project(project_name, score_type='regression')

# set up influence algorithm
tru.set_influence_type("shap")

# reduce settings for speed
tru.set_num_internal_qii_samples(100)
tru.set_num_default_influences(100)

# add data collection
tru.add_data_collection("Data Collection v1")

### After we've done so, the next thing to do is adding the data.

To do so, we'll introduce the add_data method along with the `ColumnSpec`. `add_data` is a general purpose method for adding data to truera and can be used to ingest a variety of data types including feature data, predictions, influences, labels and extra data.

`ColumnSpec` is a helper class we imported from truera.client.ingestion that we use to specify which columns in the dataframe are for what purpose. If you prefer, you can skip using `ColumnSpec` and just pass a dict instead, e.g. `{'id_col_name': 'id', ...}`

In [ ]:
# add data to the collection we just created
tru.add_data(data=san_francisco,
             data_split_name="San Francisco",
             column_spec=ColumnSpec(id_col_name="id",
                                    pre_data_col_names=list(san_francisco.columns.drop(["id", "price"])),
                                    label_col_names="price"))
tru.add_data(data=seattle,
             data_split_name="Seattle",
             column_spec=ColumnSpec(id_col_name="id",
                                    pre_data_col_names=list(seattle.columns.drop(["id", "price"])),
                                    label_col_names="price"))

After adding our data, it's a good time to set our background set for calculating feature influences. Generally, it's a good practice to set the background set equal to the training set. We'll do so here.

In [ ]:
tru.set_influences_background_data_split("San Francisco")

### Train and register the model

In [ ]:
# train first model
xgb_reg = xgb.XGBRegressor(eta=0.2, max_depth=4)
xgb_reg.fit(san_francisco.drop(['id', 'price'], axis=1), san_francisco.price)

# register the model
tru.add_python_model("model_1",
                     xgb_reg,
                     train_split_name="San Francisco",
                     train_parameters={
                         "model_type": "xgb.XGBRegressor",
                         "eta": 0.2,
                         "max_depth": 4
                     })

### Calculate and ingest predicitions.

For calculation, we'll use the built-in tru.get_ys_pred() to compute predictions.

In [ ]:
tru.set_data_split("San Francisco")
sf_preds = tru.get_ys_pred().reset_index(names="id")
tru.set_data_split("Seattle")
se_preds = tru.get_ys_pred().reset_index(names="id")

Once we've computed our predictions we can add them again using add_data. We'll also use column_spec here, but this time we need to supply our prediction_col_names.

In [ ]:
tru.add_data(data=sf_preds,
             data_split_name="San Francisco",
             column_spec=ColumnSpec(id_col_name="id", prediction_col_names="__truera_prediction__"))

tru.add_data(data=se_preds,
             data_split_name="Seattle",
             column_spec=ColumnSpec(id_col_name="id", prediction_col_names="__truera_prediction__"))

### Computing and Ingesting Feature Influences

Feature influences are a core component of the TruEra platform that enables model explainability.

To compute them, we'll use `compute_feature_influences`. We'll use opensource SHAP as our influence calculation method here. 

Note: TruEra-QII is available for paid users to provide faster and more accurate computation.

In [ ]:
tru.set_influence_type("shap")

for data_split_name in ["Seattle", "San Francisco"]:
    tru.set_data_split(data_split_name)
    tru.compute_feature_influences()

### Computing and Ingesting Error Influences

Error influences are a core component of the TruEra platform that enables model explainability.

To compute them, we'll again use `compute_feature_influences`, except with a different score type: `"mean_absolute_error_for_regression"`.

In [ ]:
for data_split_name in ["Seattle", "San Francisco"]:
    tru.set_data_split(data_split_name)
    tru.compute_feature_influences(score_type='mean_absolute_error_for_regression')

### Issue: Overfitting

We observe there to be a large discrepency between our train and test accuracy!

In [ ]:
# set default performance metric
tru.set_default_performance_metrics(["MAE"])

train_split_name = "San Francisco"
test_split_name = "Seattle"

# generate the explainer and compute performance
explainer = tru.get_explainer(test_split_name, comparison_data_splits=[train_split_name])
explainer.compute_performance(metric_type="MAE")

### Create Tests

To help us keep track of this issue, let's add a test for it!

Additionally, too many unimportant features is a common cause of overfitting. We should test for that as well.

Note that we could also set up tests for fairness and stability.

In [ ]:
# add performance tests
tru.tester.add_performance_test(test_name='Relative MAE Test',
                                all_data_collections=True,
                                data_split_name_regex='Seattle',
                                metric="MAE",
                                reference_split_name=train_split_name,
                                fail_if_greater_than=0.80,
                                fail_threshold_type="RELATIVE")

tru.tester.add_performance_test(test_name='RMSE Test',
                                all_data_collections=True,
                                data_split_name_regex='.*',
                                metric="RMSE",
                                fail_if_greater_than=115,
                                fail_threshold_type="ABSOLUTE")

### View Test Results

In [ ]:
# get model results
tru.set_model("model_1")
tru.tester.get_model_test_results(test_types=["performance"])

Both tests are failing.

### From here, navigate to the TruEra Web App for analysis or continue on to Part 2!     [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16DexGCY1i4A5fLJZXC7xHPpqCSrQhVab)